In [ ]:
import pandas as pd
import csv
import torch

df = pd.read_csv("../nasdaq_exteral_data.csv", error_bad_lines=False, warn_bad_lines=False, engine='python', encoding='utf-8')




In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers.utils import logging

logging.set_verbosity_info()

print("Downloading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
print("Tokenizer loaded.")

print("Downloading model...")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", force_download=True)
print("Model loaded.")


In [ ]:
import os

df['Date'] = df['Date'].str[:10]


df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

print(df.head())


news = df[['Date', 'Article_title', 'Stock_symbol', 'Article']]


news = news.dropna()

stock_price_dir = "../full_history"

stock_price_dfs = []

for file in os.listdir(stock_price_dir):
    symbol = file.replace(".csv", "")
    df = pd.read_csv(os.path.join(stock_price_dir, file))
    df['Stock_symbol'] = symbol
    stock_price_dfs.append(df)

stock_df = pd.concat(stock_price_dfs, ignore_index=True)

stock_df["date"] = pd.to_datetime(stock_df["date"], errors='coerce')



In [ ]:
news['Date'] = pd.to_datetime(news['Date'], errors='coerce')
stock_df['date'] = pd.to_datetime(stock_df['date'], errors='coerce')

stock_df = stock_df.rename(columns={'date' : 'Date'})

# Clear the DataFrame before merging new DataFrames
merged_df = None

news = news.sort_values('Date')
stock_df = stock_df.sort_values('Date')

merged_df = pd.merge_asof(
    news,
    stock_df,
    by="Stock_symbol",
    on="Date",
    direction="backward"
)

print(merged_df.head())

In [ ]:
print("YUH")


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
print("Model loaded???")
# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model loaded")
def get_sent_batch(texts):
    # Tokenize the input texts
    print("dawg")
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    print("gang")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    print("hi")
    # Perform inference without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
    print("hello")
    # Compute probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).tolist()
    print("yuh")
    # Determine sentiments based on probabilities
    sentiments = ["positive" if p[2] > p[0] else "negative" for p in probs]
    return sentiments

testing = "The stock market is performign well right now"
print(get_sent_batch([testing]))

print("ADOAJSDOIJ")
# Test the function with a sample text
test_text = merged_df.iloc[0]['Article_title']
print(get_sent_batch([test_text]))